##Assignment 3 Naïve Bayes and Sentiment Classification and Logistic Regression
Instructions
* Read the following Chapter 4: Naive Bayes and Sentiment Classification. Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2021. All rights reserved. Draft of September 21, 2021. I have tried to pull out relevant notes for you below, but it is encouraged that you read each chapter provided.
* Read the following Chapter 5: Logistic Regression. Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2021. All rights reserved. Draft of September 21, 2021. I have tried to pull out relevant notes for you below, but it is encouraged that you read each chapter provided.

Summary
Classification is one of the most important tasks of NLP and in machine learning. In NLP it often means the task of text categorization for both sentiment analysis, spam detection, and topic modeling. Naïve Bayes is often one of the first classification algorithms defined in NLP.  The intuition behind a classifier is lies at the underlying probability inferred by the Bayesian Inference, which uses Baye’s rule and conditional probabilities.

Here’s a reminder on Baye’s Rule:
P(y)=P(x)P(x)/(P(y))

We are saying “what is the probability of x given y”. Naïve Bayes is a generative model because there is an input that helps the model determine what the output could be. Said differently, “to train a generative model we first collect a large amount of data in some domain (e.g., think millions of images, sentences, or sounds, etc.) and then train a model to generate data like it.” [6]

So in the case of Naïve Bayes, we say given some word, what should be the class of the current word we are assessing? Contrastingly, discriminative models such as logistic regression, learn from features provided to the algorithm and then determine or predict what the class is. [7]


With Naïve Bayes, the assumption is that the probabilities are independent. We often call the Naïve Bayes classifier the bag-of-words approach. That’s because we are essentially throwing in the collection of words into a ‘bag’, selecting a word at random, and then calculating their frequency to use in the Bayesian Inference. Thus, context – the position of words -- is ignored and despite this, it turns out that the Naïve Bayes approach can be accurate and effective at determining whether an email is spam for example.

Back to bag of words. With bag of words, we assume that the position of the words are not relevant -- that dependency or context in the word phrase or sentence doesn’t matter. Relatedly, the naive Bayes assumption implies that the conditional probabilities are independent -- a rather strange assumption to make for words in a sentence! The equation for the naive Bayes classifier is outlined below:

You can use Naive Bayes by creating an index of words and walking through every word position in a test or corpus.


It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.

For this Assignment, you can start with a spam/ham dataset, then predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).   One example corpus:   https://spamassassin.apache.org/old/publiccorpus/

You may work alone or in a group on this project.  You're welcome to use any tools or approach that you like.  Due before our next meetup. Starter code provided below.

Test example is provided at the end.

Libraries you may wish to use

In [ ]:
import pandas as pd
import numpy as np
from os import makedirs, path, remove, rename, rmdir
from tarfile import open as open_tar
from shutil import rmtree
from urllib import request, parse
from glob import glob
from os import path
from re import sub
from email import message_from_file
from glob import glob
from sklearn.model_selection import StratifiedShuffleSplit
from collections import defaultdict
from functools import partial
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score)
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
import gc

download corpus using the following functions

Note: you may need to mount your drive on google then run this location. See previous exercises.

In [ ]:
def download_corpus(dataset_dir: str = 'data'):
    base_url = 'https://spamassassin.apache.org'
    corpus_path = 'old/publiccorpus'
    files = {
        '20021010_easy_ham.tar.bz2': 'ham',
        '20021010_hard_ham.tar.bz2': 'ham',
        '20021010_spam.tar.bz2': 'spam',
        '20030228_easy_ham.tar.bz2': 'ham',
        '20030228_easy_ham_2.tar.bz2': 'ham',
        '20030228_hard_ham.tar.bz2': 'ham',
        '20030228_spam.tar.bz2': 'spam',
        '20030228_spam_2.tar.bz2': 'spam',
        '20050311_spam_2.tar.bz2': 'spam' }

    #creates the folders: downloads, ham and spam
    downloads_dir = path.join(dataset_dir, 'downloads')
    ham_dir = path.join(dataset_dir, 'ham')
    spam_dir = path.join(dataset_dir, 'spam')

    makedirs(downloads_dir, exist_ok=True)
    makedirs(ham_dir, exist_ok=True)
    makedirs(spam_dir, exist_ok=True)


    for file, spam_or_ham in files.items():
        # download files from URL of each specific .bz2 file
        url = parse.urljoin(base_url, f'{corpus_path}/{file}')
        tar_filename = path.join(downloads_dir, file)
        request.urlretrieve(url, tar_filename)

        #list e-mails in the compressed .bz2 file
        emails = []
        with open_tar(tar_filename) as tar:
            tar.extractall(path=downloads_dir)
            for tarinfo in tar:
                if len(tarinfo.name.split('/')) > 1:
                    emails.append(tarinfo.name)

        # move e-mails to ham or spam directory
        for email in emails:
            directory, filename = email.split('/')
            directory = path.join(downloads_dir, directory)

            if not path.exists(path.join(dataset_dir, spam_or_ham, filename)):
                rename(path.join(directory, filename),
                   path.join(dataset_dir, spam_or_ham, filename))

        rmtree(directory)

download_corpus()

#How many e-mails are classified in our dataset as either Spam or not Spam?


In [ ]:
#How many e-mails are classified in our dataset as either Spam or not Spam?
ham_dir = path.join('data', 'ham')
spam_dir = path.join('data', 'spam')

print('Number of Non-Spam E-mails:', len(glob(f'{ham_dir}/*')))
print('\nNumber of Spam E-mails:', len(glob(f'{spam_dir}/*')))

Number of Non-Spam E-mails: 6952

Number of Spam E-mails: 2399


In [ ]:
import pandas as pd

In [ ]:
import glob

In [ ]:
ham_files = glob.glob('data/ham/*')
spam_files = glob.glob('data/spam/*')  # Replace 'data/spam/' with the actual path to your spam directory

# Create an empty list to store the email data
email_data = []

In [ ]:
# Process ham files
for file_path in ham_files:
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()
        email_data.append((content, 'ham',0))

# Process spam files
for file_path in spam_files:
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()
        email_data.append((content, 'spam',1))

In [ ]:
# Create a dataframe from the email data
df = pd.DataFrame(email_data, columns=['content', 'label','spam'])

# Now you can work with the dataframe 'df' as desired
# For example, you can print the first few rows
df.tail()

,content,label,spam
9346,From pitster262859971@hotmail.com Thu Jul 25 ...,spam,1
9347,From wclimenh@megamail.pt Mon Sep 16 13:07:44...,spam,1
9348,From hfcfyouroffers676@hotmail.com Mon Jun 24...,spam,1
9349,From bebkokori@eircom.net Mon Jun 24 17:51:10...,spam,1
9350,From cbuBrookie69@hushmail.com Mon Sep 2 16:...,spam,1


In [ ]:
# Assuming you already have the dataframe 'df' with 'content' and 'label' columns

# Shuffle the dataframe
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Now you can work with the shuffled dataframe 'shuffled_df' as desired
# For example, you can print the first few rows
shuffled_df.head()

,content,label,spam
0,From belinda_navarro@hotmail.com Mon Sep 2 1...,spam,1
1,From razor-users-admin@lists.sourceforge.net ...,ham,0
2,From mailings@ticketmaster.co.uk Mon Jun 24 1...,spam,1
3,From fork-admin@xent.com Tue Aug 20 22:30:45 ...,ham,0
4,From exmh-users-admin@redhat.com Mon Sep 2 2...,ham,0


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
# Download required NLTK resources (only once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def clean_text(text):
    # Remove email headers, metadata, footers, and signatures
    # email_pattern = r'(From:.*?Subject:|Sent:|To:|Cc:|Bcc:|Reply-To:|Return-Path:|Received:|Message-ID:|Date:|Content-Type:).*?(\n|$)'
    # text = re.sub(r"(?m)^From:.*(?:\n(?!\n).*)*", "", text)
    text = re.split(r"\n\n", text, maxsplit=1)[-1]
    # text = re.sub(email_pattern, '', text)

    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove non-text characters
    text = re.sub('[^A-Za-z]', ' ', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join tokens back into a single string
    cleaned_text = ' '.join(tokens)

    return cleaned_text

In [ ]:
# Apply text preprocessing to the 'content' column
shuffled_df['preprocessed_content'] = shuffled_df['content'].apply(clean_text)

<ipython-input-13-87a79248c79f>:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()
<ipython-input-13-87a79248c79f>:9: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [ ]:
shuffled_df.head()

,content,label,spam,preprocessed_content
0,From belinda_navarro@hotmail.com Mon Sep 2 1...,spam,1,Norton AD Take Control Your Computer With This...
1,From razor-users-admin@lists.sourceforge.net ...,ham,0,Here recipe Maildrop Vipul Razor check log usr...
2,From mailings@ticketmaster.co.uk Mon Jun 24 1...,spam,1,Creamfields Saturday th August pm licence gran...
3,From fork-admin@xent.com Tue Aug 20 22:30:45 ...,ham,0,Gordon Mohr wrote Does mean Eliptic Curve encr...
4,From exmh-users-admin@redhat.com Mon Sep 2 2...,ham,0,If already enable debug log Hacking Support pr...


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Assuming you have the shuffled dataframe 'shuffled_df' with 'content' and 'label' columns

# Perform text preprocessing (cleaning, lowercasing, etc.)

# Feature extraction using Bag-of-Words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(shuffled_df['content'])
y = shuffled_df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the classifier (Naive Bayes example)
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Predict labels for the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1408
        spam       0.98      0.94      0.96       463

    accuracy                           0.98      1871
   macro avg       0.98      0.97      0.98      1871
weighted avg       0.98      0.98      0.98      1871



Provide your classifier below

In [ ]:
##### START CODE ####

""" Enter code here"""

#### END CODE ####

The following email is a test email. You can take this and test your classifier to see if it predicts spam or not.



In [ ]:
spam_email = """
Subject: Get Rich Quick!

Dear Friend,

Congratulations! You've been selected to participate in an exclusive opportunity to make thousands of dollars from the comfort of your own home. Our revolutionary system guarantees quick and easy cash with minimal effort.

No more struggling to pay bills or worrying about financial security. With our proven method, you can start earning massive amounts of money in no time.

Here's what some of our satisfied customers have to say:
- "I was skeptical at first, but I'm now living my dream life thanks to this incredible system!" - John S.
- "I never thought making money online could be this simple. It's changed my life!" - Sarah L.

Don't miss out on this limited-time offer. Act now to secure your spot and start enjoying a life of financial freedom.

Click the link below to get started:
www.getrichquick.com

Remember, this opportunity is exclusive and won't last long. Take control of your financial future today!

Best regards,
The Get Rich Quick Team
"""


In [ ]:
test = clean_text(spam_email)
testing = vectorizer.fit_transform([test])
testing

<1x87 sparse matrix of type '<class 'numpy.int64'>'
	with 87 stored elements in Compressed Sparse Row format>

In [ ]:
# Predict the label for the email
prediction = classifier.predict(X[-1:][0])
print(prediction[0])

spam
